In [1]:
from statsbombpy import sb

import os

import pandas as pd

import warnings
warnings.filterwarnings("ignore")

email = "nathan.talbot@etu.uca.fr"
password = os.environ["mdp_statsbomb"]
creds = {"user" : email, "passwd" : password}

In [3]:
cmpet = sb.competitions(creds = creds)

In [5]:
cmpet[cmpet.competition_name == "Ligue 2"]

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
48,8,281,France,Ligue 2,male,False,False,2023/2024,2024-06-15T21:47:35.765201,2024-07-09T12:40:17.906144,2024-07-09T12:40:17.906144,2024-06-15T21:47:35.765201
49,8,235,France,Ligue 2,male,False,False,2022/2023,2024-03-29T17:05:56.986423,2024-06-11T09:09:20.548934,2024-06-11T09:09:20.548934,2024-03-29T17:05:56.986423
50,8,108,France,Ligue 2,male,False,False,2021/2022,2024-06-12T22:24:59.036894,2024-06-12T22:25:53.057195,2024-06-12T22:25:53.057195,2024-06-12T22:24:59.036894
51,8,90,France,Ligue 2,male,False,False,2020/2021,2024-06-25T09:39:53.886416,2024-06-25T09:45:50.084168,2024-06-25T09:45:50.084168,2024-06-25T09:39:53.886416


In [11]:
events = sb.competition_events(country = "France", division = "Ligue 2", season = "2023/2024", gender = "male", creds = creds)

In [2]:
dico_annee = {"2023_2024" : "2023/2024", "2022_2023" : "2022/2023", "2021_2022" : "2021/2022", "2020_2021" : "2020/2021"}

for annee in dico_annee.keys() :
    print(annee)
    events = sb.competition_events(country = "France", division = "Ligue 2", season = dico_annee[annee], gender = "male", creds = creds)

2023_2024


In [15]:
events_grouped = sb.competition_events(country = "France", division = "Ligue 2", season = "2023/2024", gender = "male", split = True, creds = creds)